# TP ESTIMATION

**Elève 1** : Mohamed Ben ALI


## Import des fonctions

In [11]:
# Importing libraries
import numpy as np
from scipy.stats import norm
from scipy.stats import uniform
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture


In [12]:
# Probability density of a Gaussian mixture at a point
def gm_pdf(x, mu, sigma, p):
    # Initialization of the output variable
    result = 0.0
    # Check the consistency of the input parameters
    # The mean vector must have the same length as the probability vector p
    if len(mu) != len(p):
        print('Dimension error in the mean vector')
    # The standard deviation vector must have the same length as the probability vector p
    elif len(sigma) != len(p):
        print('Dimension error in the standard deviation vector')
    else:
        # Calculation of the density value
        for i in range(0, len(p)):
            result = result + p[i] * norm.pdf(x, mu[i], sigma[i])
    return result


In [13]:
# Generation of random numbers following a mixture of Gaussians
def gm_rnd(mu, sigma, p):
    # Initialization of the output variable
    result = 0.0
    # Check the consistency of the input parameters
    # The mean vector must have the same length as the probability vector p
    if len(mu) != len(p):
        print('Dimension error in the mean vector')
    # The standard deviation vector must have the same length as the probability vector p
    elif len(sigma) != len(p):
        print('Dimension error in the standard deviation vector')
    else:
        # Sample generation
        # A sample is drawn from a uniform distribution on [0,1]
        u = uniform.rvs(loc=0.0, scale=1.0, size=1)
        # Each subsequent test defines an interval where the probability
        # of the uniform variable belonging to it matches one of the probabilities
        # defined in the probability vector p. If u belongs to one of these intervals,
        # it is equivalent to having generated a random variable according to one of the
        # elements of p. For example, in the first test below, the probability of u
        # being in the interval [0, p[0]] is equal to p[0] since u follows a uniform
        # distribution. Therefore, if u belongs to [0, p[0]], it is equivalent to
        # drawing according to the event with probability p[0].
        if u < p[0]:  # Check if an event with probability p[0] was generated
            result = sigma[0] * norm.rvs(loc=0, scale=1, size=1) + mu[0]
            # To generate a random variable following any normal distribution, simply
            # multiply a standard normal variable (mean 0 and standard deviation 1)
            # by the desired standard deviation and add the desired mean to the result.
        for i in range(1, len(p)):
            if (u > np.sum(p[0:i])) and (u <= np.sum(p[0:i+1])):  # Check if an event
                # with probability p[i] was generated
                result = sigma[i] * norm.rvs(loc=0.0, scale=1.0, size=1) + mu[i]
                # To generate a random variable following any normal distribution, simply
                # multiply a standard normal variable (mean 0 and standard deviation 1)
                # by the desired standard deviation and add the desired mean to the result.
    return result


In [14]:
def EM_algorithm(nbMaxIterations, mu_0, sigma_0, alpha_0, donnees):

    # Initialization values
    mu_em = mu_0
    sigma_em = sigma_0
    alpha_em = alpha_0  # The sum must equal 1

    nbIteration = 1  # Initialization of the stopping variable
    nbComposante = len(alpha_em)  # Number of mixture components
    nbDonnees = len(donnees)  # Number of data points
    p = np.zeros(shape=(nbComposante, nbDonnees))
    # Declaration and initialization of the matrix that will hold probabilities
    # p(k|x, current_theta)

    alpha_em_new = alpha_em
    sigma_em_carre_new = sigma_em
    mu_em_new = mu_em
    donneesP = np.zeros(shape=(nbDonnees))

    while nbIteration < nbMaxIterations:
        # Expectation step: calculate probabilities
        for n in range(0, nbDonnees, 1):
            for k in range(0, nbComposante, 1):
                p[k, n] = alpha_em[k] * norm.pdf(x=donnees[n], loc=mu_em[k], scale=sigma_em[k])
            p[:, n] = p[:, n] / np.sum(p[:, n])  # Normalize probabilities

        # Maximization step: update parameters
        for k in range(0, nbComposante, 1):
            alpha_em_new[k] = np.sum(p[k, :]) / nbDonnees  # Update weights
            for n in range(0, nbDonnees, 1):
                donneesP[n] = donnees[n] * p[k, n]
            mu_em_new[k] = np.sum(donneesP) / np.sum(p[k, :])  # Update means
            for n in range(nbDonnees):
                donneesP[n] = ((donnees[n] - mu_em_new[k]) ** 2) * p[k, n]
            sigma_em_carre_new[k] = np.sum(donneesP) / np.sum(p[k, :])  # Update variances
        mu_em = mu_em_new
        sigma_em = np.sqrt(sigma_em_carre_new)  # Convert variances to standard deviations
        alpha_em = alpha_em_new
        nbIteration = nbIteration + 1

    return np.array([mu_em, sigma_em, alpha_em])


## Part 1: Testing the EM Algorithm on Synthetic Data
After testing the EM algorithm on synthetic data, the following observations were made:

Initial Values Impact: Changing the initial values can significantly influence the estimation. Choosing initial values that are "close" to the true values leads to better estimations. Therefore, it is preferable to determine an approximate range for the desired parameters before using the EM algorithm.

Effect of the Number of Iterations: With a good choice of initial parameters, the number of iterations seems to have little effect on the estimation. However, selecting too few iterations can prevent the algorithm from converging properly.

## Part 2: Galaxies
We will attempt to estimate the distribution of the values contained in galaxies.txt as a Gaussian mixture model. Before we begin, we need to determine the number of Gaussians in the mixture, and for each of them, we must set an initialization for:

Its mean $\mu_0$
Its standard deviation $\sigma_0$
Its "probability" $\alpha_0$
To simplify the visualization of the data and avoid potential effects caused by the large values of velocities, we will center and scale the data. In the two figures below, we will first display the data as a scatter plot, and then as a histogram.






4o mini

In [ ]:
x_galaxies = np.loadtxt("galaxies.txt")
x_galaxies = (x_galaxies - np.mean(x_galaxies)) / np.std(x_galaxies) 
nb_samples = len(x_galaxies)

plt.plot(x_galaxies, 'g.')
plt.title('Distribution des vitesses')
plt.xlabel('Indice')
plt.ylabel('Vitesse')
plt.grid()
plt.show()

def plot_galaxy_hist():
    plt.hist(x_galaxies, bins = 30, density = True, edgecolor = "black")
    plt.title('Distribution des vitesses')
    plt.xlabel('Vitesse')
    plt.ylabel("Nombre d'échantillons")

plot_galaxy_hist()
plt.show()

## Comment

In the first step, we will focus on the main trends we can observe in these two plots. Three distinct groups seem to emerge, so we will choose a mixture of three Gaussians. For the initialization values, we calculate the mean and standard deviation of three groups, selected for now "by eye" by observing the curves:

Between the first and the 8th value
Between the 8th and the 76th value
And from the 76th value to the last one.

In [ ]:
# Chose the velues
batches = np.split(x_galaxies, [8,76])
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = np.array([0.2, 0.7, 0.1])
nb_iterations = 40

In [ ]:
# Execution
resultat = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)
mu_em, sigma_em, alpha_em = resultat

def print_parameters(mu, sigma, alpha):
    print('Les paramètres estimés sont : ')
    print('Moyennes des composantes du mélange', mu)
    print('Ecart type des composantes du mélange', sigma)
    print('Probabilités des composantes du mélange', alpha)
    
print_parameters(mu_em, sigma_em, alpha_em)

In [ ]:
# Display distribution

x = np.linspace(-3, 3, 10000)
def plot_distribution_comparison(estimation):
    plot_galaxy_hist()
    plt.plot(x, estimation, 'r-', label = 'Estimée')
    plt.legend(loc='upper left', shadow=True, fontsize='x-large')

estimation_3_components = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation_3_components)
plt.show()

## Comment

Overall, the estimated distribution follows the trends of the empirical distribution. However, for values between -1 and 1, it seems that the estimation could be more accurate. In particular, two nearby local maxima appear to be distinct, but they are not differentiated by our initial estimator.

In [ ]:
#  Initializing

batches = np.split(x_galaxies, [8,40,50])
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = np.array([0.2, 0.4, 0.3, 0.1])

nb_iterations = 40
resultat = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)
mu_em, sigma_em, alpha_em = resultat

estimation_4_components = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation_4_components)
plt.show()

print_parameters(mu_em, sigma_em, alpha_em)

## Comment
With a mixture of 4 Gaussians, the two local maxima in the middle are better represented, but the quality of the prediction for less probable velocities is reduced (particularly for large velocities, where a distinct peak was observed with 3 Gaussians).

We can attempt to improve this model by refining the clustering of the values. To do this, we will use the K-means method via sklearn.

In [ ]:
def kmeans_cluster(data, n_clusters):
    kmeans = KMeans(n_clusters = n_clusters, random_state=0).fit(data.reshape(-1,1))
    labeled_data = np.stack((kmeans.labels_, data), axis =1) # zip data with the label
    labeled_data = labeled_data[labeled_data[:,0].argsort()] # order by label
    return np.split(labeled_data[:,1], np.unique(labeled_data[:, 0], return_index = True)[1][1:]) # split values based on labels

In [ ]:
batches = kmeans_cluster(x_galaxies, 4)
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = np.array([0.2, 0.4, 0.3, 0.1])

In [ ]:
nb_iterations = 40
resultat = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)
mu_em, sigma_em, alpha_em = resultat

estimation_4_components = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation_4_components)
plt.show()

print_parameters(mu_em, sigma_em, alpha_em)

## Commentaire
By applying this clustering method, we observe that the estimation is closer to the empirical distribution in the "less probable" cases: we recover the peak around 2.75 and reduce the error for value sets with zero density. However, the two peaks around 0 are less well differentiated: the distribution is more similar to our case with 3 Gaussians, albeit more accurate.

Therefore, we need a method to more intelligently choose the number of Gaussians. In the following, we attempt to use the BIC (Bayesian Information Criterion) and AIC (Akaike Information Criterion) to determine the optimal number of Gaussians.

In [ ]:
n = 20    
bics = []
aics = []
for i in range(1, n):
    
    gmm = GaussianMixture(n_components=i)
    gmm.fit(x_galaxies.reshape(-1,1))
    bics.append(gmm.bic(x_galaxies.reshape(-1,1)))
    aics.append(gmm.aic(x_galaxies.reshape(-1,1)))
    
    
plt.plot(np.arange(1,n),bics, label = 'BIC')
plt.plot(np.arange(1,n),aics, label = 'AIC')
plt.legend(loc='best')
plt.show()

print(np.argmin(bics), np.argmin(aics))
    

## Comment

The BIC and AIC criteria help favor parsimony, meaning a balance between the quality of the estimation and the number of model parameters. Based on the calculation above, we determine that the number of components in the mixture should be 2 according to the BIC criterion, and more than 10 according to the AIC criterion. The BIC penalizes the number of model parameters more than the AIC, which partially explains this difference.

It is important to note that, with each execution of the cell above, the minimum values of AIC and BIC can change (BIC typically takes values of 2 and 4, while AIC can vary between 12 and 18).

Now, let's perform the estimation in both of these cases.

In [ ]:
def default_alpha_0(size):
    return np.full((1,size), 1/size)[0]

# Pour 2 gaussiennes
batches = kmeans_cluster(x_galaxies, 2)
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = default_alpha_0(2)

mu_em, sigma_em, alpha_em = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)

estimation = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation)
plt.show()

# Pour 10 gaussiennes
batches = kmeans_cluster(x_galaxies, 10)
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = default_alpha_0(10)

mu_em, sigma_em, alpha_em = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)

estimation = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation)
plt.show()

## Comment

First, we note that we limited the second estimation to 10 Gaussians, as beyond that, the results did not seem to display correctly.

We observe that by using the BIC, the estimated model is simple, capturing only the most distinctive variations in the data. In contrast, using 10 Gaussians leads to a very complex model that attempts to account for all apparent variations in the data.

We would likely prefer an intermediate number of Gaussians. Perhaps it would be beneficial to find another criterion more suited to the problem in order to determine the optimal number of Gaussians. Additionally, a criterion is needed to assess the effectiveness of the model without prior knowledge of the initial model.

In [ ]:
# Pour 5 gaussiennes
batches = kmeans_cluster(x_galaxies, 5)
mu_0 = np.array([np.mean(batch) for batch in batches])
sigma_0 = np.array([np.std(batch) for batch in batches])
alpha_0 = default_alpha_0(5)

mu_em, sigma_em, alpha_em = EM_algorithm(nb_iterations, mu_0, sigma_0, alpha_0, x_galaxies)

estimation = gm_pdf(x, mu_em, sigma_em, alpha_em)    
plot_distribution_comparison(estimation)
plt.show()